In [1]:
# necessary import statements

import neo4j

import pandas as pd

from IPython.display import display

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [5]:
def my_neo4j_nodes_relationships():
    "print all the nodes and relationships"
   
    print("-------------------------")
    print("  Nodes:")
    print("-------------------------")
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    display(df)
    
    print("-------------------------")
    print("  Relationships:")
    print("-------------------------")
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    display(df)
    
    density = (2 * number_relationships) / (number_nodes * (number_nodes - 1))
    
    print("-------------------------")
    print("  Density:", f'{density:.1f}')
    print("-------------------------")

In [95]:
# assumes that MovieGraph database is set up in Neo4j using other notebooks
# use this to double check you are querying the correct graph
my_neo4j_nodes_relationships()

-------------------------
  Nodes:
-------------------------


,node_name,labels
0,(no genres listed),[genres]
1,Action,[genres]
2,Adventure,[genres]
3,Animation,[genres]
4,Children's,[genres]
...,...,...
10366,193581,[movie]
10367,193583,[movie]
10368,193585,[movie]
10369,193587,[movie]


-------------------------
  Relationships:
-------------------------


,node_name_1,node_1_labels,relationship_type,node_name_2,node_2_labels
0,1,[user],RATED,1,[movie]
1,1,[movie],RATEDBY,1,[user]
2,1,[user],RATED,3,[movie]
3,1,[movie],RATEDBY,5,[user]
4,1,[user],RATED,6,[movie]
...,...,...,...,...,...
201667,193581,[movie],RATEDBY,184,[user]
201668,193583,[movie],RATEDBY,184,[user]
201669,193585,[movie],RATEDBY,184,[user]
201670,193587,[movie],RATEDBY,184,[user]


-------------------------
  Density: 0.0
-------------------------


In [131]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'user', {relationshipProperties: ANY})
session.run(query)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Procedure call does not provide the required number of arguments: got 2 expected at least 3 (total: 4, 1 of which have default values).

Procedure gds.graph.project has signature: gds.graph.project(graphName :: STRING?, nodeProjection :: ANY?, relationshipProjection :: ANY?, configuration  =  Map{} :: MAP?) :: nodeProjection :: MAP?, relationshipProjection :: MAP?, graphName :: STRING?, nodeCount :: INTEGER?, relationshipCount :: INTEGER?, projectMillis :: INTEGER?
meaning that it expects at least 3 arguments of types STRING?, ANY?, ANY?
Description: Creates a named graph in the catalog for use by algorithms. (line 1, column 1 (offset: 0))
"CALL gds.graph.project('ds_graph', 'user')"
 ^}

In [130]:
query = """

CALL gds.degree.stream('ds_graph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as degree
ORDER BY degree DESC, name

"""

my_neo4j_run_query_pandas(query)

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: Invalid relationship projection, one or more relationship types not found: 'average_rating'}